In [11]:
import requests
import json
import os
from dotenv import load_dotenv
import subprocess

load_dotenv()
KAFKA_PATH = os.getenv('KAFKA_PATH')

Activate Kafka-Connector

In [17]:
KAFKA_CONNECT_URL = 'http://localhost:8083'


def create_hdfs_connector(topic_name):
    config = {
        "name": f"hdfs-sink-{topic_name}",
        "config": {
            "connector.class": "io.confluent.connect.hdfs.HdfsSinkConnector",
            "topics": topic_name,
            "hdfs.url": "hdfs://localhost:9000",
            "flush.size": "1000",
            "format.class": "io.confluent.connect.hdfs.json.JsonFormat",
            "schema.compatibility": "NONE",
            "value.converter": "org.apache.kafka.connect.json.JsonConverter",
            "value.converter.schemas.enable": "false",
            "consumer.auto.offset.reset": "earliest" #Read existing data
        }
    }
    
    with open(f'{topic_name}-connector.json', 'w') as f:
        json.dump(config, f)
    
    # Create connector
    try:
        subprocess.run([
            'curl', '-X', 'POST',
            f'{KAFKA_CONNECT_URL}/connectors',
            '-H', 'Content-Type: application/json',
            '-d', f'@{topic_name}-connector.json'
        ], check=True)
        print(f"Connector for '{topic_name}' created")
    except subprocess.CalledProcessError as e:
        print(f"Failed to create connector for '{topic_name}', {e}")
        
create_hdfs_connector('superstore')

Connector for 'superstore' created
